In [1]:
# !python3 -m pip install torch==1.8.2+cu102 torchvision==0.9.2+cu102 torchaudio==0.8.2 -f https://download.pytorch.org/whl/lts/1.8/torch_lts.html
# !python3 -m pip install simpletransformers
# !python3 -m pip install pandas scikit-learn
# !python3 -m pip install transformers-interpret

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import confusion_matrix, classification_report,f1_score
import numpy as np
from scipy.spatial.distance import cdist
import heapq
from collections import Counter
from simpletransformers.classification import ClassificationModel,ClassificationArgs
from simpletransformers.classification import MultiLabelClassificationModel

EVALMODE=0

In [2]:
df_train = pd.read_csv("../data/classification/train/train_complex_aug.tsv",sep="\t")
df_train= df_train.rename(columns={'0': 'text',"1":"labels"})
df_train.drop("multi",axis=1,inplace=True)
df_train


,text,labels
0,movie vara level la Erika poguthu,0
1,movie beethi level la pole mairu,1
2,movie .... sanghu kulla mokka poguthu,1
3,movie vara da asinga Erika fucking,1
4,Padam nalla comedy padama irukum polaye..,0
...,...,...
109955,Just WTF WORST Bangam jaldra shit,1
109956,நாமக்கல் மாவட்டம் வன்னியர் சார்பாக திரௌபதி பட...,0
109957,punda மாவட்டம் வன்னியர் pooolu shit mokka போடா...,1
109958,நாமக்கல் மாவட்டம் வன்னியர் Dai திரௌபதி flop e ...,1


In [3]:
df_test = pd.read_csv("../data/classification/test/test_tamil_offensive_binary.tsv",sep="\t")
df_test

,text,labels
0,14.12.2018 epo trailer pathutu irken ... Semay...,0
1,Paka thana poro movie la Enna irukunu,0
2,Suriya anna vera level anna mass,0
3,suma kaththaatha da sound over a pooda kudaath...,1
4,ஆனவக் கொலைகள் நடக்காவிடில் ...நம் அடையாளம் அளி...,1
...,...,...
4227,Vijay going to next level thalaivaaaaa,0
4228,மண்ணு பொண்ணு ரெண்டுமே ஒன்னு அதுல எவன் கைய வச்ச...,1
4229,asuran= aadukalam+pudupettai+ wada chennai..ye...,0
4230,Vijay's all movies look like same.,0


# Classification Training (New labels)

In [4]:
# train_using_new_label

In [5]:
if EVALMODE==0:
    model_args = ClassificationArgs(
        num_train_epochs=5,
        overwrite_output_dir=True, 
        manual_seed=42,
        max_seq_length=150,
        #sliding_window=True,
        eval_batch_size=64,
        best_model_dir = "models/best_xlmroberta_complex_aug",
        do_lower_case=True,
        early_stopping_patience=2,
        save_model_every_epoch=False,
        early_stopping_consider_epochs=True,
        use_early_stopping=True,
        loss_type= "focal",
        evaluate_during_training=True,
        #wandb_project='ST-Hope-CELoss',
        learning_rate=3e-05)
    model = ClassificationModel(
        model_type='xlmroberta',
        model_name='xlm-roberta-base', 
        use_cuda=torch.cuda.is_available(),
        cuda_device="0",
        num_labels=2, 
        args=model_args)

#roberta roberta-base
#xlmroberta xlm-roberta-base
#bert bert-multilingual-base-uncased     

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense

In [6]:
if EVALMODE==0:
    model.train_model(df_train,args={'train_batch_size':64},eval_df=df_test)
elif EVALMODE==1:
    model_args = ClassificationArgs(
        eval_batch_size=64)
    model = ClassificationModel(
        model_type='bert',
        model_name='../models/offensive_bert_simple_aug/', 
        use_cuda=torch.cuda.is_available(),
        cuda_device="0", args=model_args)

/home/ubuntu/.local/lib/python3.6/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 0 of 5:   0%|          | 0/1719 [00:00<?, ?it/s]

/home/ubuntu/.local/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


  0%|          | 0/4232 [00:00<?, ?it/s]

Running Epoch 1 of 5:   0%|          | 0/1719 [00:00<?, ?it/s]

  0%|          | 0/4232 [00:00<?, ?it/s]

  0%|          | 0/4232 [00:00<?, ?it/s]

In [7]:
result, model_outputs, wrong_predictions = model.eval_model(df_test)
print(result)


  0%|          | 0/4232 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/67 [00:00<?, ?it/s]

{'mcc': 0.4757941880384622, 'tp': 675, 'tn': 2698, 'fp': 492, 'fn': 367, 'auroc': 0.8491859156793964, 'auprc': 0.6099437969296222, 'eval_loss': 0.14554149123714932}


In [8]:
testx = [i for i in df_test["text"].values.tolist()]
testx
predictions, raw_outputs = model.predict(testx)

ypr=[]
for i in predictions:
        ypr.append(i)

ygt=[]
for i in df_test["labels"].values:
        ygt.append(i)

print(classification_report(ygt,ypr,digits=5))

  0%|          | 0/4232 [00:00<?, ?it/s]

  0%|          | 0/67 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0    0.88026   0.84577   0.86267      3190
           1    0.57841   0.64779   0.61114      1042

    accuracy                        0.79702      4232
   macro avg    0.72933   0.74678   0.73690      4232
weighted avg    0.80594   0.79702   0.80074      4232



In [1]:
# Interpret Predictions

In [11]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers_interpret import SequenceClassificationExplainer
import pandas as pd
import os
from sklearn.metrics import classification_report
import numpy as np
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

df_test = pd.read_csv("../data/classification/test/test_tamil_offensive_binary.tsv",sep="\t")
df_test

model_name = "../models/offensive_bert_complex_aug"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

model     = model.to('cuda:0')


In [14]:
testx = [i for i in df_test["text"].values.tolist()]
cls_explainer = SequenceClassificationExplainer(
        model,
        tokenizer)

ygt=[]
for i in df_test["labels"].values:
        ygt.append(i)
        
ypr=[]     
count=0
attribution_list_complex=[]
for sntce in testx:
    print(sntce)
    word_attributions = cls_explainer(sntce)
    ypr.append(cls_explainer.predicted_class_index)
    attribution_list_complex.append(word_attributions)

print(classification_report(ygt,ypr,digits=5))

14.12.2018 epo trailer pathutu irken ... Semaya iruku
Paka thana poro movie la Enna irukunu
Suriya anna vera level anna mass
suma kaththaatha da sound over a pooda kudaathu pa s3 1 month oda stop aakidum then bairavaa da aaddchi than kaththi kaththi thondaiya kilikatha pa
ஆனவக் கொலைகள் நடக்காவிடில் ...நம் அடையாளம் அளிக்கப்படும்... இயக்குனருக்கு வாழ்த்துக்கள்...தமிழ்நாடு முத்தரையர்கள்
teasere intha alavukk masss aa irunthal padam enthalavukku masssa iruppano.... Ithukkum mele. Luv u Suriya anna ur die hard fan from kerala
Enthanai kudumbam velila sollamudiyama sethukutu iruku... Irukura unmaye sollum..... Padam. Dharmapuri veera Venniyer sarbaga valthukal...
Power ranger megazord niyabagam vantha hit like
Parkavakula Udayar, moopanar, nainar sarbhaka valthukal
Part 2 எடுத்து எடுத்து நல்ல படைப்பான part 1 ன தரம் குறைக்காதீர்கள்........2018 ல part 2 எதுவும் ஓடல நல்லாவும் இல்ல அப்பறம் 2019 இப்போ 2020
Na oru 1000 times mela pathu irukn
1st Ayan super hit 2nd maatran mega hit  3rd kaapaan kan

Theme pathuttu  vanthavanga like ah thattunga
Udhay ku kannu theriyadho indha movie la🤔
SEMA MASSS தலைவா i am your big fan A.K.வெங்கடேஷ் பிரசாத்,DEEE ஜில்லா மக்கள் இயக்கத் தலைவர் ஆவாரங்குப்பம்
Trailer Ditto kekka bekka🤐 expected more
Thala pola varuma  Thala eppavume mass vera leval
Thala Vera level Seekiram vanga Thala
semma mass bgm . i am waiting thalaivaaaa
விஸ்வகர்மா கம்மாளர் சங்கம் சார்பாக படம் வெற்றிபெற வாழ்த்துக்கள்
I love you chiyan Vikram Anna Marana masss
Devidiya punda...  eva yaru da gommala
இது உண்மை ...திருமாவுக்கு சரியான செருப்பு அடி........ இதே தேவேந்திரகுல வேளாளர் மக்கள் வரவேற்கிறோம்......
தமிழ் சினிமாவை உலகதறத்தில் கொண்டு போகும் ஓறீரு இயக்குனரில் இவர் ஒருவர்  தளமான படமாக psycho  இருக்கும் என்பது உருதி
Mugamoodi part 2
THALAIVAR FAN SK KU WISHES THALAIVAR VERIYAN
Bro ethu apdiye kekka bekka short film mariye eruku seriyana copy bro
Rip  Sr Prabhu punda mavan... Ne theru Kodi thaan di...  Sad luck Karthi and Lokesh Anna...
இரண்டு படங்களின் Trailer பார்த்த மாதிறி இருக்க

சாதியின் பெயரால் எடுக்கப்பட்ட படம் நல்லா இருக்கோ இல்லையோ. ஆனால் சாதியின் பெயரால் எடுக்கப்படும் இந்த மாதிரி திரைப்படங்களால்  அந்தந்த கால கட்டத்தில் இருக்கும் இளைஞர்கள் மனதில் சாதி வெறியை சுலபமாக பதிய வைத்து விடுகிறார்கள். இது எல்லா சாதியினருக்கும் பொருந்தும்.
0.45 ishwarya Menon kulla Samantha etti paakranga
Politics pathi padam na kandipa naan pappen, Like if you watch this movie 🤗🤗🤗
பொன்னியின் செல்வன் Now reach 3M.  Rocking and trending..
Dhanush Vetrimaran Gv  Yathuku pa promosion panununum Panamala  akum Movie  Don't worry
Verithanama iruku suriya anna yuvan bgm vera leval selva sir unngala romba romba tamil cinema miss pannuthu sir
Class padathukku ivlo mass look..just Thala things
Ayoo trisha vjs vida thalaivar simran young ah irkangaaaaaaa...attitude vera level
ஜெயம் ரவி மறுபடியும் ஒரு புதிய கதை களத்துடன் இந்த படமும் வெற்றி பெற வாழ்த்துக்கள் ரவி அண்ணா
un like pandravan cinmava pakka kutathu..yenna avan oru rasanai Ellathavan theri semma mass
Sema vera level ma. Love u so much
Ann

surya Anna s3 marana mass teaser
Vjs fans hit like like
Thala naa mass  Mass naa Thala
இதில்  தலீத் சமுதாயம் தவிர மற்ற அனைத்து சமுதாய மக்களும் பாதிக்கப்பட்டு ஆதரவு கொடுக்குராங்களை ஆதரவுக்கு நன்றி  வெற்றி! வெற்றி ! வெற்றி !
Ajith ku set agala.. No mass scene
Verithanama irukum Mannu ponnu rendum mukkiyam sema
I like kaappaan more than ngk
Vera level nanpa. Padam vetri adaiya vendukiren. Sila koottathukku adi verthurukkum
Anna dialogu lam vera level na mass NJK
Probably a super-hit movie.. Script s very unique..
September 8 ( Today )...13 Years Of Sillunu Oru Kaadhal .
Padam disaster by Thala fan 🤧
Very mass . Mass varanum rajini fans hit like
Trailer Semmaiya irukku Pakka Blockbuster  Party Time Thalaaaa
Thala machan richid entha padam hit aakanum
thala sema as usual , hard worker pa kathukanum avarkita irundhu
YOU WILL SEE MY RAGE
Vaai vittu sirital noai vittu pogum
Petta 713K LIKES Nerkonda paarvai 721K LIKES
STR   நிங்கள் இல்லாமல் நான் இல்லை ....
ஆயுத எழுத்து மாதிரி இருக்கே எனக்கு மட

Thala fans hit like me....
Superrrrrrrrrrrrrrrr star na super star than maaaasssssssss
As,str,av,vs, bangam ar musical Vera level mani sir movie ahh ethu
Udayanidhi fans like here ... Likes Varuma??  Fans irupaangala ...
yaru ellam thala oda work dedication eh vfx effects nu solli criticize panrangalo , 1) unga kitta adhu kana proof edhuvum illa 2) avaru vandhu  day and night ah hard work panni nammakulukaga padam panraru adhuku konjam atleast respect kuduka kathukonga summa veetula ukandhitu kora solra nala nammu onnum periya manishan agida mudiyathu
Yaarellam padam paatha aprom trailer paakura
Niraiya niraiya niraiya niraiya irukku...NKP update .. waiting
Saw Movie maathiri iruku... But anyways Myskin  waiting badly
Enna Hari padam trailer paatha maari irukku...🤫
Vera level தல Going to be a biggest blockbuster
Iniku may 31 padam realease aairchu
..tha vfx  vera level da yebhaaa
Yuvan 2k kids ayum addicted aga vachutaru.
Surya + Selva + u1 = perfect combo
கோத்தா செம படம் போல... கண்டிப

...

Baatsha style la yae irukku paaa
2009 AYAN   2019 KAAPPAAN  Vera level movies
மற்ற சமூகத்தில் யாரும் எந்த பெண்களையும் ஏமாற்ற வில்லையா குறிப்பிட்ட சமூகத்தை மட்டும் குறிப்பிட்டு எடுப்பதற்கான காரணம் என்ன தவறு அனைத்து சமூக இளைஞர்களும் செய்கின்றனர் ஏதோ இவர்கள் எல்லாம் ஒன்றும் தெரியாத குழைந்தகள் போலவும் குறிப்பிட்ட சமூகம் மட்டும்தான் தவறு செய்வது போலவும் ஒரு பிம்பத்தை ஏற்படுத்துகின்றனர் உண்மையாக காதலித்து திருமணம் செய்து கொண்டாலும் ஆணவகொலை செய்து வாழ விடாமல்  உங்கள் ஜாதி வெறியை தீர்த்து கொள்கின்றனர் நீங்கள் சொல்லும் குறிப்பிட்ட சமூகத்தில் உண்மையாக காதலித்து திருமணம் செய்து கொண்டு வாழத்தான் செய்கின்றனர் மற்ற சமூகத்தில் நாடக காதல் செய்யவில்லையா இங்கு காதலிப்பது பிரச்சனையா இல்லை அவர்கள் உங்கள் பெண்களை காதலிப்பது பிரச்சனையா
From Andhra,  I love myskin movies
Draupathi Mohan G sema Anna
Thala na mass uh...thala fans like podu
comments paakum pothu thaamba theriuthu ethana jaathi irukunu 🤣🤣🤣🤣
திருநெல்வேலி....... நாடார் சமுதாய மக்களின் சார்பாக படம் வெற்றி பெற வாழ்த்துக்கள்........
Maranam.maasu mar

#வரலாம்  #வரலாம் வா #பைரவா   #விளையாடலாமா
mass na vjiay vjiayna mass bythalafans
raja Rani - mouna ragam  theri - satriyan
Str fans vj fans treat erku
கடலூர் மாவட்ட பிராமண சமூக மக்கள் சார்பாக இத்திரைப்படம் வெற்றிபெற வாழ்த்துக்கள்.  இயக்குனர் திரு.G.மோகன் அவர்களுக்கும் எங்கள் நன்றி கலந்த வணக்கங்கள்.
seri kadaisiya varrathu rajiniyaa illa ms baskar aah
கேஸ் போடுரவங்களுக்கு ஒன்னு சொல்றேன் படத்தை படமா பாருங்கட.படத்தை பார்த்துட்டு கேஸ் போடுங்கடா முட்டா பசங்க மாரி டிரெய்லரை பார்த்துட்டு கேஸ் போடாதிங்கடா....
Dei ingaum evanavathu seemanku support panni vanthiga na suicide pannikiven..
UYa hooo my name is Narayanaa
Always super star mass thalivaa naa unga kuda eppoma irupan
intha padam pudikala na enna sollurathu nu theriyala
Nalla per irunthale padam psyco mari irukum ithula pere psycona epudi irukummmm. We r waiting miskin sir
Jai surya jai jai surya, proud to be surya fan
தர்மபுரி மாவட்டம் வன்னியர் சார்பாக படம் வெற்றி பெற வாழ்த்துக்கள் மோகன் அண்ணா கோடான கோடி நன்றி இனி நம்ம பொண்ண பார்க்க கூட

Hit the like button especially for the final dialogue.. யோகி பாபு : இது 2016 நம்பளனா இத பாரு.  ர‌ஜினி: நான் அரசியலுக்கு வருவது உறுதி. ஜெயம் ரவி:  ஏய் யாரா ஏமாத்த பாக்கற இது 96.  Bangam yaa.. 🤣🤣
Poda Sunni tavidiya talapathi tha Gath ponda sexe  opp ponda
Padippu Mattum nammakitte yirunthu edukkamidiyathu Big salute
I'm just now watching this movie Romba Nalaiku Appuram Aiyo Adhukkulla Padam Mudinjuruchana Feel Panna Vacha Movie... After Kgf
கோயம்பத்தூர் சிங்ககுட்டிக்கு கிடைக்க போகுது ஹாட்ரிக் வெற்றிவாழ்த்துக்கள் ஹிப்ஹாப் தமிழா ஆதி அண்ணா🧡🧡
Good arjun sarja fans hit like here
ஜெயம்ரவி இன்னொரு ஹிட் படம் தமிழில் புது விதமான படங்கள் நீங்கள் அளிப்பது மிகவும் சந்தோஷமாக உள்ளது  தமிழில் ஸ்வாம்பி, விண்வெளி போன்ற புது கதை களங்கள் இன்னும் நிறைய எதிர்பார்க்கிறோம்
Aiyooooo.......inna lookuda love you Thalaiva
pub g thumbnail nala iruku onum vitu vaikathinga
Who's here after visvasam trailer ?
Yen paa hip hop uu un padathula heroine laam nalla jammunu irukkaangalae…
NAYAKKAR VAMSAM SARBAGA INDHA MOVI

திருமா இனி மேல் நீ குருமா
Bigil ku yevlo wait panramo athe alavuku kaithiku waiting
இயற்கையாவே உற்பத்தி ஆகற நதிய தனக்கு மட்டுந்தானு சொந்தம் கொண்டாட்ற உரிமைய யாருங்க Remember hydrocarbon, methane issue
நாமக்கல் மாவட்டம் கொங்கு வேளாள கவுண்டர்கள் சார்பாக படம் வெற்றி பெற வாழ்த்துக்கள்
Padam pathuttu vanthu yaarellam intha trailer paaka vantheenga
Caste pakkama Unmaya love panravangalum irukka thana seiranga, love panra paiyan nallavana family nalla family ah nu paarunga pa, ellam ok na inter caste mrg panrathu thappu illa thana... Ithu Nalla unmaya love panravangalum paathika paduvangaleeee...
indha trailer paakumbodhu enakkum andha disorder vandhrumo nu irukudhu pa
1.40 ku U1 konnudaru bgm la  bgme ivelo gethu nu annana intha bgm la epdi kattuvaru selva varattum pakkkkkkka arasiyal....
Hindi la PK (Aamir khan) movie maadhiri vithyaasamaana storyline. Epdi use pannirukinga nu therila. Trailer super. Waiting for movie.
யாருசாமி இவன்.... சினிமா துறையில் எத்தனையோ இயக்குநர்கள் இருக்கின்றனர் B

KeyboardInterrupt: 

In [ ]:
attribution_list_complex = np.array(attribution_list_complex)

In [ ]:
np.save("../models/attribution_complex.npy",attribution_list_complex)